In [15]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
from pyscan.drivers.testing.auto_test_driver import test_driver
from pyvisa import ResourceManager, VisaIOError
import pytest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            print(r, name)
            break
    except VisaIOError:
        pass
    res.close()

GPIB0::8::INSTR Stanford_Research_Systems,SR830,s/n86813,ver1.07 



In [4]:
name

'Stanford_Research_Systems,SR830,s/n86813,ver1.07 \n'

In [5]:
srs830 = ps.Stanford830(res)

test_driver(srs830)

dict_keys(['instrument', '_driver_class', 'debug', '_instrument_driver_version', '_version', 'black_list_for_testing', '_id_settings', '_phase_settings', '_reference_source_settings', '_frequency_settings', '_reference_slope_settings', '_harmonic_settings', '_amplitude_settings', '_input_configuration_settings', '_input_ground_settings', '_input_coupling_settings', '_input_line_filter_settings', '_sensitivity_settings', '_reserve_mode_settings', '_time_constant_settings', '_filter_slope_settings', '_synchronous_filter_settings', '_display1_output_source_settings', '_display2_output_source_settings', '_auxillary_voltage1_settings', '_auxillary_voltage2_settings', '_auxillary_voltage3_settings', '_auxillary_voltage4_settings', '_sample_rate_settings', '_end_buffer_mode_settings', '_trigger_mode_settings', '_id', '_phase', '_reference_source', '_frequency', '_reference_slope', '_harmonic', '_amplitude', '_input_configuration', '_input_ground', '_input_coupling', '_input_line_filter', '_se

AssertionError: no valid type present in setting: _id_settings. Must be one of ['values', 'range', 'ranges', 'indexed_values', 'dict_values'].

In [17]:
# Test get and set display

srs830.set_display(1, 'r', 'aux1') # number, source, ratio
source, ratio = srs830.get_display(1)
assert source == 'r'
assert ratio == 'aux1'


srs830.set_display(1, 'x', 'none')
source, ratio = srs830.get_display(1)
assert source == 'x'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(1, 'y', 'none')
print('Sucessfully failed setting channel 1 to y')
    
    
srs830.set_display(2, 'theta', 'aux3')
source, ratio = srs830.get_display(2)
assert source == 'theta'
assert ratio == 'aux3'


srs830.set_display(2, 'y', 'none')
source, ratio = srs830.get_display(2)
assert source == 'y'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(2, 'x', 'none')
print('Sucessfully failed setting channel 2 to x')

print("Successfully tested get and set display")


Sucessfully failed setting channel 1 to y
Sucessfully failed setting channel 2 to x
Successfully tested get and set display


In [7]:
# Test get and set offset and expand

srs830.set_channel_offset_expand(1, 100, 10)
print(srs830.get_channel_offset_expand(1))

srs830.set_channel_offset_expand(1, 0, 1)
print(srs830.get_channel_offset_expand(1))


['0', '1']
['0', '0']


In [31]:
# Test auto offset
# v0 should be some value, v_offset should be close to 0
srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('x')
srs830.auto_offset('x')
v_offset = srs830.read('x')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(2, 0, 1)
v0 = srs830.read('y')
srs830.auto_offset('y')
v_offset = srs830.read('y')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001

srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('r')
srs830.auto_offset('r')
v_offset = srs830.read('r')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(1, 0, 1)
srs830.set_display(1, 'x', 'none')


2.18428e-09 2.18428e-09
-2.18439e-09 -2.18439e-09
2.18428e-09 2.18428e-09


In [34]:
# Test read aux inputs
# shoudl show some random small value
for i in range(1, 5):
    aux_input = srs830.read_aux_input(i)
    print(aux_input)
    assert abs(aux_input - 0) < 0.1

0.00166667
0.004
0.001
-0.0176667


In [10]:
# Read x, y, r, theta

print(srs830.read('x'))
print(srs830.read('y'))
print(srs830.read('r'))
print(srs830.read('theta'))

2.18428e-09
-2.18439e-09
2.18428e-09
-2656.6929484751354


In [35]:
# Read display

for i in range(1, 3):
    print(srs830.read_display(i))

2.18439e-09
-2.18451e-09


In [39]:
# Test snap

for source in ['y', 'r', 'theta', 'aux1', 'aux2', 'aux3',
                  'aux4', 'frequency', 'display1', 'display2']:
    print(srs830.snap('x', source))

srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1')


with pytest.raises(AssertionError):
    srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1', 'display2')
    # snap only accepts 6 entries

[2.18428e-09, -2.18439e-09]
[2.18428e-09, 2.18428e-09]
[2.18428e-09, -2638.129100232685]
[2.18428e-09, 0.002]
[2.18428e-09, 0.001]
[2.18428e-09, 0.00166667]
[2.18428e-09, 0.0116667]
[2.18428e-09, 1000.0]
[2.18428e-09, 2.18439e-09]
[2.18428e-09, -2.18451e-09]


In [14]:
# test get ID

id = srs830.get_identificaiton_string()
assert id == 'Stanford_Research_Systems,SR830,s/n86813,ver1.07 '
print(id)

Stanford_Research_Systems,SR830,s/n86813,ver1.07 


In [95]:
# test buffer_points

srs830.buffer_points
srs830.wait_for_trigger()
srs830.set_buffer_mode(0.25)
srs830.wait_for_trigger()
# srs830.start()
srs830.trigger()
sleep(20)
srs830.buffer_points


5

In [91]:
srs830.query('SRAT?')

'0\n'

In [92]:
srs830.sample_rate

0.0625